# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [67]:
import pandas as pd

In [68]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']

data = pd.read_csv(r'C:\Users\btdjf\Desktop\Ironhack 2\Lab NLP\lab-nlp\your-code/training.1600000.processed.noemoticon.csv', names=column_names, encoding='latin-1')
data

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [69]:
data = data.sample(n=20000)
data

,target,id,date,flag,user,text
1495821,4,2069999955,Sun Jun 07 16:55:18 PDT 2009,NO_QUERY,yokomarie510,"On my way up to [his] job, he called talk`n bo..."
2388,0,1468367488,Tue Apr 07 01:14:20 PDT 2009,NO_QUERY,joacimhagg,@theresev solen var inge varm
1227054,4,1990937963,Mon Jun 01 06:04:20 PDT 2009,NO_QUERY,Kumiyo_,In one of those sick moods yaaaaayyy
534907,0,2197731667,Tue Jun 16 15:04:56 PDT 2009,NO_QUERY,linnanm,Got home from Trondheim 2night! We had so much...
626580,0,2230639116,Thu Jun 18 17:23:49 PDT 2009,NO_QUERY,elchupahueso,Got a B in my skills class. Very disappointed...
...,...,...,...,...,...,...
1314935,4,2013971796,Wed Jun 03 00:05:26 PDT 2009,NO_QUERY,azaxacavabanama,@iruleatwork the hindi one i guess. was browsi...
367932,0,2049351547,Fri Jun 05 16:43:48 PDT 2009,NO_QUERY,MarkSheppard,"@ElaineGiles We don't buy our Macs, we just re..."
79158,0,1751707334,Sat May 09 20:05:03 PDT 2009,NO_QUERY,Joannethecat,"@roccoman so sad please be safe, me luvs U ro..."
1550493,4,2183795447,Mon Jun 15 14:59:51 PDT 2009,NO_QUERY,a_l_wells,mock the week baybieeee


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [70]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

In [71]:
def clean_up(text):
    cleaned_text = re.sub('[^a-zA-Z]', ' ', text)
    cleaned_text = re.sub(r'http\S+', '', cleaned_text)
    cleaned_text = cleaned_text.lower()
    return cleaned_text

In [72]:
def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

In [73]:
def stem_and_lemmatize(words):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_and_lemmatized = []

    for word in words:
        stemmed_word = stemmer.stem(word)
        lemmatized_word = lemmatizer.lemmatize(stemmed_word)
        stemmed_and_lemmatized.append((lemmatized_word))

    return stemmed_and_lemmatized

In [74]:
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words and len(word) > 1]
    return filtered_tokens

In [75]:
data['text_processed'] = data['text'].apply(lambda x: remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x)))))

In [76]:
data

,target,id,date,flag,user,text,text_processed
1495821,4,2069999955,Sun Jun 07 16:55:18 PDT 2009,NO_QUERY,yokomarie510,"On my way up to [his] job, he called talk`n bo...","[way, hi, job, call, talk, bout, quot, momma, ..."
2388,0,1468367488,Tue Apr 07 01:14:20 PDT 2009,NO_QUERY,joacimhagg,@theresev solen var inge varm,"[theresev, solen, var, ing, varm]"
1227054,4,1990937963,Mon Jun 01 06:04:20 PDT 2009,NO_QUERY,Kumiyo_,In one of those sick moods yaaaaayyy,"[one, sick, mood, yaaaaayyy]"
534907,0,2197731667,Tue Jun 16 15:04:56 PDT 2009,NO_QUERY,linnanm,Got home from Trondheim 2night! We had so much...,"[got, home, trondheim, night, much, fun, got, ..."
626580,0,2230639116,Thu Jun 18 17:23:49 PDT 2009,NO_QUERY,elchupahueso,Got a B in my skills class. Very disappointed...,"[got, skill, class, veri, disappoint, mean, ne..."
...,...,...,...,...,...,...,...
1314935,4,2013971796,Wed Jun 03 00:05:26 PDT 2009,NO_QUERY,azaxacavabanama,@iruleatwork the hindi one i guess. was browsi...,"[iruleatwork, hindi, one, guess, wa, brow, thr..."
367932,0,2049351547,Fri Jun 05 16:43:48 PDT 2009,NO_QUERY,MarkSheppard,"@ElaineGiles We don't buy our Macs, we just re...","[elainegil, buy, mac, rent, even, get, use, mu..."
79158,0,1751707334,Sat May 09 20:05:03 PDT 2009,NO_QUERY,Joannethecat,"@roccoman so sad please be safe, me luvs U ro...","[roccoman, sad, plea, safe, luv, roccoman, hea..."
1550493,4,2183795447,Mon Jun 15 14:59:51 PDT 2009,NO_QUERY,a_l_wells,mock the week baybieeee,"[mock, week, baybieee]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [77]:
from nltk import FreqDist

In [78]:
all_words = [word for tokens in data['text_processed'] for word in tokens]
all_words

['way',
 'hi',
 'job',
 'call',
 'talk',
 'bout',
 'quot',
 'momma',
 'bore',
 'quot',
 'sooo',
 'go',
 'hi',
 'entertain',
 'ili',
 'daddi',
 'theresev',
 'solen',
 'var',
 'ing',
 'varm',
 'one',
 'sick',
 'mood',
 'yaaaaayyy',
 'got',
 'home',
 'trondheim',
 'night',
 'much',
 'fun',
 'got',
 'cold',
 'though',
 'got',
 'skill',
 'class',
 'veri',
 'disappoint',
 'mean',
 'need',
 'serious',
 'focu',
 'next',
 'quarter',
 'le',
 'zealotri',
 'knife',
 'skill',
 'wait',
 'till',
 'th',
 'june',
 'jonassss',
 'enjoy',
 'pre',
 'lucki',
 'peopl',
 'wait',
 'month',
 'befor',
 'sprint',
 'give',
 'enough',
 'discount',
 'upgrad',
 'prelaunch',
 'read',
 'quot',
 'untitl',
 'continu',
 'quot',
 'sleepi',
 'time',
 'stuf',
 'face',
 'pasta',
 'red',
 'velvet',
 'cake',
 'amp',
 'ice',
 'cream',
 'sick',
 'stomach',
 'headach',
 'wish',
 'someon',
 'could',
 'come',
 'rub',
 'templ',
 'rain',
 'drop',
 'keep',
 'fallin',
 'head',
 'omg',
 'censor',
 'omg',
 'sexiest',
 'men',
 'aliv',
 'ce

In [79]:
freq_dist = FreqDist(all_words)
freq_dist

FreqDist({'go': 1681, 'get': 1446, 'wa': 1330, 'day': 1312, 'thi': 1214, 'good': 1161, 'work': 1096, 'like': 1093, 'love': 1026, 'got': 964, ...})

In [80]:
top_words = freq_dist.most_common(5000)
top_words

[('go', 1681),
 ('get', 1446),
 ('wa', 1330),
 ('day', 1312),
 ('thi', 1214),
 ('good', 1161),
 ('work', 1096),
 ('like', 1093),
 ('love', 1026),
 ('got', 964),
 ('quot', 905),
 ('http', 877),
 ('today', 844),
 ('one', 796),
 ('time', 793),
 ('lol', 768),
 ('thank', 765),
 ('back', 726),
 ('miss', 714),
 ('want', 687),
 ('know', 679),
 ('think', 668),
 ('realli', 655),
 ('feel', 635),
 ('com', 634),
 ('im', 627),
 ('amp', 623),
 ('see', 611),
 ('night', 582),
 ('well', 582),
 ('need', 578),
 ('na', 549),
 ('oh', 545),
 ('still', 540),
 ('new', 529),
 ('come', 528),
 ('watch', 525),
 ('hope', 524),
 ('home', 518),
 ('look', 511),
 ('make', 500),
 ('ha', 452),
 ('twitter', 448),
 ('last', 446),
 ('much', 435),
 ('morn', 433),
 ('tomorrow', 423),
 ('great', 417),
 ('wait', 411),
 ('sleep', 406),
 ('haha', 403),
 ('fun', 393),
 ('wish', 383),
 ('whi', 371),
 ('sad', 367),
 ('week', 353),
 ('follow', 352),
 ('right', 345),
 ('tri', 345),
 ('thing', 343),
 ('friend', 338),
 ('would', 338),
 

In [81]:
selected_words = [word for word, frequency in top_words]
selected_words

['go',
 'get',
 'wa',
 'day',
 'thi',
 'good',
 'work',
 'like',
 'love',
 'got',
 'quot',
 'http',
 'today',
 'one',
 'time',
 'lol',
 'thank',
 'back',
 'miss',
 'want',
 'know',
 'think',
 'realli',
 'feel',
 'com',
 'im',
 'amp',
 'see',
 'night',
 'well',
 'need',
 'na',
 'oh',
 'still',
 'new',
 'come',
 'watch',
 'hope',
 'home',
 'look',
 'make',
 'ha',
 'twitter',
 'last',
 'much',
 'morn',
 'tomorrow',
 'great',
 'wait',
 'sleep',
 'haha',
 'fun',
 'wish',
 'whi',
 'sad',
 'week',
 'follow',
 'right',
 'tri',
 'thing',
 'friend',
 'would',
 'gon',
 'onli',
 'happi',
 'sorri',
 'say',
 'way',
 'tonight',
 'bit',
 'bad',
 'nice',
 'tweet',
 'though',
 'hi',
 'veri',
 'even',
 'take',
 'start',
 'better',
 'peopl',
 'bed',
 'yeah',
 'could',
 'hour',
 'guy',
 'school',
 'play',
 'use',
 'show',
 'hey',
 'twitpic',
 'rain',
 'cant',
 'awesom',
 'ye',
 'next',
 'hate',
 'soon',
 'weekend',
 'find',
 'lt',
 'never',
 'let',
 'sick',
 'year',
 'final',
 'plea',
 'yay',
 'littl',
 'a

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [82]:
def create_feature_set(document, selected_words):
    document_words = set(document)
    features = {}
    for word in selected_words:
        features[word] = (word in document_words)
    
    return features
feature_set = []
for index, row in data.iterrows():
    document = row['text_processed']
    sentiment = row['target']
    features = create_feature_set(document, selected_words)
    feature_set.append((features, sentiment))

for features, sentiment in feature_set:
    data = data.sample(10000)
features, sentiment

({'go': True,
  'get': False,
  'wa': False,
  'day': False,
  'thi': False,
  'good': False,
  'work': False,
  'like': False,
  'love': False,
  'got': False,
  'quot': False,
  'http': True,
  'today': False,
  'one': False,
  'time': False,
  'lol': False,
  'thank': False,
  'back': False,
  'miss': False,
  'want': False,
  'know': False,
  'think': False,
  'realli': False,
  'feel': False,
  'com': True,
  'im': False,
  'amp': True,
  'see': False,
  'night': False,
  'well': False,
  'need': False,
  'na': False,
  'oh': False,
  'still': False,
  'new': False,
  'come': False,
  'watch': False,
  'hope': False,
  'home': False,
  'look': False,
  'make': False,
  'ha': False,
  'twitter': False,
  'last': False,
  'much': False,
  'morn': False,
  'tomorrow': False,
  'great': False,
  'wait': False,
  'sleep': False,
  'haha': False,
  'fun': False,
  'wish': False,
  'whi': False,
  'sad': False,
  'week': False,
  'follow': False,
  'right': False,
  'tri': False,
  'thin

# From here forward optional 

### Building and Traininng Naive Bayes Model

    In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
# your code here

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here